In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix, f1_score, classification_report, plot_confusion_matrix
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC

In [401]:
# pulled in dataframe
df = pd.read_csv('../../src/data/syriatel_customer_churn.csv')


In [ ]:
# dropped area code and phone number. still looking into if area code should be included
df = df.drop(['area code', 'phone number'], axis = 1)

In [ ]:
X = df.drop('churn', axis=1)
y = df['churn']

In [ ]:
# Saving dataframes to src/data folder to be able to use in any notebook. 
#X.to_csv('../../src/data/X_dataframe.csv')
#y.to_csv('../../src/data/y_dataframe.csv')

In [ ]:
# train-test-split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Our first model should have accuracy above 86 percent.
dummy_model = DummyClassifier(strategy="most_frequent")
dummy_model.fit(X_train, y_train)
y_hat_dummy = dummy_model.predict(X_test)

In [ ]:
y_test.value_counts()

In [ ]:
print(dummy_model.score(X_test, y_test))
print(confusion_matrix(y_test, y_hat_dummy))
print(classification_report(y_test, y_hat_dummy))

In [ ]:
def One_hot(X):
    '''
    Returns data frame of X_train and X_test of catagorical columns one 
    hot encoded merged with continous columns. 
    Catagorical anything that is an object and continous is not object
    '''
    X_num = X.select_dtypes(exclude='object') # numerical cols
    X_cat = X.select_dtypes(include='object') # categorical cols
    
    ohe = OneHotEncoder(drop='first', sparse=False)
    ohe.fit(X_cat)

    X_catagory = pd.DataFrame(ohe.transform(X_cat),
                                      columns=ohe.get_feature_names(X_cat.columns),
                                        index=X.index)
    X_ohe = X_num.merge(X_catagory,
                           left_index=True,
                          right_index=True)
    


    return X_ohe, X_catagory

In [ ]:
X_train_ohe, X_train_catagory = One_hot(X_train)

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_train_ss = pd.DataFrame(ss.fit_transform(X_train_ohe), index= X_train_ohe.index, columns= X_train_ohe.columns)

# Smote

In [ ]:
# for reproducibility purposes
sm = SMOTE(random_state=42)


# SMOTE number of neighbors


X_resample, y_resample = sm.fit_resample(X_train_ss, y_train)

In [ ]:
y_resample.value_counts()

In [ ]:
X_test_ohe, X_test_catagory = One_hot(X_test)

In [ ]:
ss = StandardScaler()
ss.fit(X_train_ohe)
X_test_ss = pd.DataFrame(ss.transform(X_test_ohe), index= X_test_ohe.index, columns= X_test_ohe.columns)

# Log first model

In [405]:
log1 = LogisticRegression(random_state=42)

log1.fit(X_resample, y_resample)
y_hat_log1 = log1.predict(X_test_ss)


print(log1.score(X_test_ss, y_test))
print(confusion_matrix(y_test, y_hat_log1))
print(classification_report(y_test, y_hat_log1))

0.7841726618705036
[[557 152]
 [ 28  97]]
              precision    recall  f1-score   support

       False       0.95      0.79      0.86       709
        True       0.39      0.78      0.52       125

    accuracy                           0.78       834
   macro avg       0.67      0.78      0.69       834
weighted avg       0.87      0.78      0.81       834



In [ ]:
grid_log = {
    'C': np.logspace(-3, 3, 20),
}

gs_log = GridSearchCV(estimator=LogisticRegression(), param_grid=grid_log, cv=3)

In [ ]:
gs_log.fit(X_resample, y_resample)

In [ ]:
gs_log.best_params_

In [ ]:
gs_log.best_score_

In [406]:
log2 = LogisticRegression(C = 233.57214690901213, random_state=42)
log2.fit(X_resample, y_resample)
y_hat_log2 = log2.predict(X_test_ss)

print(log1.score(X_test_ss, y_test))
print(confusion_matrix(y_test, y_hat_log2))
print(classification_report(y_test, y_hat_log2))

0.7841726618705036
[[556 153]
 [ 28  97]]
              precision    recall  f1-score   support

       False       0.95      0.78      0.86       709
        True       0.39      0.78      0.52       125

    accuracy                           0.78       834
   macro avg       0.67      0.78      0.69       834
weighted avg       0.87      0.78      0.81       834



Did about the same when setting a C value. 

# Random Forest 1st model

In [ ]:
forest = RandomForestClassifier(random_state=42)
forest.fit(X_resample, y_resample)
y_hat_forest = forest.predict(X_test_ss)

print(forest.score(X_test_ss, y_test))
print(confusion_matrix(y_test, y_hat_forest))
print(classification_report(y_test, y_hat_forest))

# Grid with Random forest

In [ ]:
grid_forest = {
    'max_features': ['sqrt', 'log2', 0.5],
    'criterion': ['gini', 'entropy'],
    'n_estimators' : [50, 100, 200]
}

In [ ]:
gs_for = GridSearchCV(estimator=RandomForestClassifier(), param_grid=grid_forest, cv=3)

In [ ]:
gs_for.fit(X_resample, y_resample)

In [ ]:
gs_for.best_params_

In [ ]:
gs_for.best_score_

In [ ]:
cross_val_score()

In [ ]:
forest2 = RandomForestClassifier(max_features=.5, random_state=42)
forest2.fit(X_resample, y_resample)
y_hat_forest2 = forest2.predict(X_test_ss)

print(cross_val_score(estimator=RandomForestClassifier(max_features=.5, random_state=42), X= X_resample, y = y_resample, cv = 3).mean())
print(forest2.score(X_test_ss, y_test))
print(confusion_matrix(y_test, y_hat_forest2))
print(classification_report(y_test, y_hat_forest2))

# KNN with Grid

In [ ]:
grid_knn = {
    'n_neighbors': list(range(3,31,2)),
    'p' : [1,2]
}

In [ ]:
gs_knn = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=grid_knn, cv=5)

In [ ]:
gs_knn.fit(X_resample, y_resample)

In [ ]:
gs_knn.best_params_

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3, p=1)
knn.fit(X_resample, y_resample)
y_hat_knn = knn.predict(X_test_ss)

print(forest2.score(X_test_ss, y_test))
print(confusion_matrix(y_test, y_hat_knn))
print(classification_report(y_test, y_hat_knn))

### Worse than just forest by itself

# SVM model

In [ ]:
svm = SVC(random_state=42)
svm.fit(X_resample, y_resample)
svm.score(X_test_ss, y_test)

In [ ]:
y_hat_svm = sr.predict(X_test_ss)

print(svm.score(X_test_ss, y_test))
print(confusion_matrix(y_test, y_hat_svm))
print(classification_report(y_test, y_hat_svm))

# Adaboost model

In [ ]:
abc = AdaBoostClassifier(random_state=42)

abc.fit(X_resample, y_resample)

y_hat_abc = sr.predict(X_test_ss)

print(abc.score(X_test_ss, y_test))
print(confusion_matrix(y_test, y_hat_abc))
print(classification_report(y_test, y_hat_abc))

# Stacking

In [ ]:
estimators = [
    ('svm', svm),
    ('knn', knn),
    ('forest', forest2)
]

sr = StackingClassifier(estimators)

In [ ]:
sr.fit(X_resample, y_resample)

In [ ]:
y_hat_sr = sr.predict(X_test_ss)

print(sr.score(X_test_ss, y_test))
print(confusion_matrix(y_test, y_hat_sr))
print(classification_report(y_test, y_hat_sr))

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 6))
fig.tight_layout(pad=4.0)

axes[0, 0].set_title("Logistic")
axes[0, 1].set_title("Logistic hyper")
axes[0, 2].set_title("Forest")
axes[1, 0].set_title("Forest hyper")
axes[1, 1].set_title("KNN")
axes[1, 2].set_title("Stack")

plot_confusion_matrix(log1, X_test_ss, y_test, ax=axes[0, 0], cmap="Blues")
plot_confusion_matrix(log2, X_test_ss, y_test, ax=axes[0, 1], cmap="Blues")
plot_confusion_matrix(forest, X_test_ss, y_test, ax=axes[0, 2], cmap="Blues")
plot_confusion_matrix(forest2, X_test_ss, y_test, ax=axes[1, 0], cmap="Blues")
plot_confusion_matrix(knn, X_test_ss, y_test, ax=axes[1, 1], cmap="Blues")
plot_confusion_matrix(sr, X_test_ss, y_test, ax=axes[1, 2], cmap="Blues")
;

In [ ]:
def plot_feature_importances(model):
    n_features = X_resample.shape[1]
    plt.figure(figsize=(8,15))
    plt.barh(range(n_features), model.feature_importances_, align='center') 
    plt.yticks(np.arange(n_features), X_resample.columns.values) 
    plt.xlabel('Feature importance')
    plt.ylabel('Feature')

In [ ]:
plot_feature_importances(forest2)

Looks like customer service calls, international plan, voice mail plan, total days charge, total day minutes are the biggest influence on this model. 

# International Plan Churn Rate

In [ ]:
churn_rate_int_plan = df.groupby(['international plan'])['churn'].mean()

fig, ax = plt.subplots(figsize =(14,10))
sns.barplot(x = churn_rate_int.index, y = churn_rate_int.values, ax = ax, palette = "Blues_d")

plt.title('Churn Rate For International Plan Holders', fontsize = 20)
plt.xlabel('')
plt.ylabel('Churn Rate', fontsize = 20)

ax.tick_params(labelsize = 20)
ax.set_xticklabels(['No International Plan', 'Has International Plan'])
plt.tight_layout()


# Customer Service calls Churn rate

In [ ]:
churn_rate_cust_service = df.groupby(['customer service calls'])['churn'].mean()

fig, ax = plt.subplots(figsize =(14,10))
sns.barplot(x = churn_rate_cust_service.index, y = churn_rate_cust_service.values, ax = ax, palette = "Blues_d")

plt.title('Churn Rate Per Number of Customer Service Calls', fontsize = 30)
plt.xlabel('Number of Customer Service Calls', fontsize = 25)
plt.ylabel('Churn Rate', fontsize = 25)

ax.tick_params(labelsize = 20)
plt.tight_layout()

In [ ]:
df.head(2)

In [ ]:
df['new'] = df['total day charge'] / df['total day minutes']

In [ ]:
df['new'].describe()

# Total day charge

In [ ]:
df2_day_charge = df[['total day charge','churn']].copy()

In [ ]:

df2_day_charge['bins'] = pd.cut(df2_day_charge['total day charge'],
                              bins=list(range(0,70,10)),
                              labels=list(range(1,7)))

In [ ]:
day_charge = df2_day_charge.groupby(['bins'])['churn'].mean()

In [ ]:
fig, ax = plt.subplots(figsize =(14,10))
sns.barplot(x = day_charge.index, y = day_charge.values, ax = ax, palette = "Blues_d")

plt.title('Churn Rate by Cost of Total Day Charge', fontsize = 30)
plt.xlabel('Cost of Total Day Charge Binned', fontsize = 25)
plt.ylabel('Churn Rate', fontsize = 25)
ax.set_xticklabels(['0-9', '10-19', '20-29','30-39', '40-49', '50-59'])

ax.tick_params(labelsize = 20)
plt.tight_layout()

# Final model

In [ ]:
grid_forest2 = {
    'max_features': ['sqrt', 'log2', 0.5, .3, None],
    'n_estimators' : [50, 100, 200, 300]
}
gs_for2 = GridSearchCV(estimator=RandomForestClassifier(), param_grid=grid_forest2, cv=3)

In [ ]:
gs_for2.fit(X_resample, y_resample)

In [ ]:
gs_for2.best_params_

In [407]:
forest_final = RandomForestClassifier(n_estimators=500, max_features=.5, random_state=42)
forest_final.fit(X_resample, y_resample)
y_hat_forest3 = forest_final.predict(X_test_ss)


In [408]:
print(cross_val_score(estimator=RandomForestClassifier(n_estimators=500, max_features=.5, random_state=42), X= X_resample, y = y_resample, cv = 3).mean())
print(forest_final.score(X_test_ss, y_test))
print(confusion_matrix(y_test, y_hat_forest3))
print(classification_report(y_test, y_hat_forest3))

0.9451219871270161
0.9460431654676259
[[688  21]
 [ 24 101]]
              precision    recall  f1-score   support

       False       0.97      0.97      0.97       709
        True       0.83      0.81      0.82       125

    accuracy                           0.95       834
   macro avg       0.90      0.89      0.89       834
weighted avg       0.95      0.95      0.95       834



# Pipeline